In [1]:
import torch
from PIL import Image
import torchvision
import torch.nn as nn
from torchvision import transforms
import torch.nn.functional as F
from torch.utils.data import Dataset, DataLoader
import numpy as np

In [80]:
class CustomDataset(Dataset):
    def __init__(self, dataset_path, image_count, fake_exif_labels): 

        self.dataset_path = dataset_path
        self.image_count = image_count
        self.transforms = transforms.Compose([
         	transforms.CenterCrop(100),
            transforms.ToTensor()])

    def __getitem__(self, index):
        image = Image.open(self.dataset_path+str(index)+".jpg").convert('RGB')
        
        return self.transforms(image), fake_exif_labels[index]

    def __len__(self):
        return self.image_count

In [81]:
batch_size = 10
temperature = 0.5
training_images_count = 10

fake_exif_labels = torch.rand(training_images_count, 10)


train_data = CustomDataset('dataset/cvFinalData/train/', training_images_count, fake_exif_labels)
train_data_loader = DataLoader(train_data, batch_size=BATCH_SIZE, shuffle=True)

In [82]:

class EXIF(nn.Module):
    def __init__(self, encoder, n_features, projection_dim):
        super(EXIF, self).__init__()

        self.encoder = encoder
        self.n_features = n_features

        self.projector = nn.Sequential(
            
            nn.Linear(self.n_features, 300, bias=False),
            nn.ReLU(),
#             nn.Dropout(0.5),
            nn.Linear(300, projection_dim, bias=False),
            nn.ReLU()
        )

    def forward(self, x):
        x = self.encoder(x)
        x = self.projector(x)
        x = F.normalize(x, dim=1)

        return x

In [85]:
encoder = torchvision.models.resnet50(pretrained=False)
n_features = 1000
projection_dim = 10
device = 'cpu'
model = EXIF(encoder, n_features, projection_dim)

In [86]:
optimizer = torch.optim.SGD(model.parameters(), lr=0.0001)
loss_function = nn.MSELoss(reduction='sum')
epochs = 1000

for epoch_counter in range(epochs):
        loss_sum = 0
        counter = 0
        print('Epoch: ', epoch_counter)
        for xs, ys in train_data_loader:
            optimizer.zero_grad()
            exif_pred = model(xs)
            loss = loss_function(exif_pred, ys)
            loss_sum += loss
            loss.backward()
            optimizer.step()
            counter += 1
            
            if counter % 1 == 0:
            	print("Training ",counter * batch_size,"/", training_images_count, " Loss: ", loss_sum / (batch_size))
            	loss_sum = 0
            

Epoch:  0
Training  10 / 10  Loss:  tensor(2.8077, grad_fn=<DivBackward0>)
Epoch:  1
Training  10 / 10  Loss:  tensor(2.0298, grad_fn=<DivBackward0>)
Epoch:  2
Training  10 / 10  Loss:  tensor(1.5618, grad_fn=<DivBackward0>)
Epoch:  3
Training  10 / 10  Loss:  tensor(1.5511, grad_fn=<DivBackward0>)
Epoch:  4
Training  10 / 10  Loss:  tensor(1.5533, grad_fn=<DivBackward0>)
Epoch:  5
Training  10 / 10  Loss:  tensor(1.5325, grad_fn=<DivBackward0>)
Epoch:  6
Training  10 / 10  Loss:  tensor(1.5273, grad_fn=<DivBackward0>)
Epoch:  7
Training  10 / 10  Loss:  tensor(1.5563, grad_fn=<DivBackward0>)
Epoch:  8
Training  10 / 10  Loss:  tensor(1.5117, grad_fn=<DivBackward0>)
Epoch:  9
Training  10 / 10  Loss:  tensor(1.5029, grad_fn=<DivBackward0>)
Epoch:  10
Training  10 / 10  Loss:  tensor(1.5305, grad_fn=<DivBackward0>)
Epoch:  11
Training  10 / 10  Loss:  tensor(1.5051, grad_fn=<DivBackward0>)
Epoch:  12
Training  10 / 10  Loss:  tensor(1.5154, grad_fn=<DivBackward0>)
Epoch:  13
Training  1

Training  10 / 10  Loss:  tensor(1.0812, grad_fn=<DivBackward0>)
Epoch:  109
Training  10 / 10  Loss:  tensor(1.0804, grad_fn=<DivBackward0>)
Epoch:  110
Training  10 / 10  Loss:  tensor(1.0659, grad_fn=<DivBackward0>)
Epoch:  111
Training  10 / 10  Loss:  tensor(1.0524, grad_fn=<DivBackward0>)
Epoch:  112
Training  10 / 10  Loss:  tensor(1.1029, grad_fn=<DivBackward0>)
Epoch:  113
Training  10 / 10  Loss:  tensor(1.0828, grad_fn=<DivBackward0>)
Epoch:  114
Training  10 / 10  Loss:  tensor(1.0761, grad_fn=<DivBackward0>)
Epoch:  115
Training  10 / 10  Loss:  tensor(1.1035, grad_fn=<DivBackward0>)
Epoch:  116
Training  10 / 10  Loss:  tensor(1.1303, grad_fn=<DivBackward0>)
Epoch:  117
Training  10 / 10  Loss:  tensor(1.0986, grad_fn=<DivBackward0>)
Epoch:  118
Training  10 / 10  Loss:  tensor(1.0935, grad_fn=<DivBackward0>)
Epoch:  119
Training  10 / 10  Loss:  tensor(1.0778, grad_fn=<DivBackward0>)
Epoch:  120
Training  10 / 10  Loss:  tensor(1.1430, grad_fn=<DivBackward0>)
Epoch:  121

Training  10 / 10  Loss:  tensor(1.0425, grad_fn=<DivBackward0>)
Epoch:  216
Training  10 / 10  Loss:  tensor(1.0504, grad_fn=<DivBackward0>)
Epoch:  217
Training  10 / 10  Loss:  tensor(1.0550, grad_fn=<DivBackward0>)
Epoch:  218
Training  10 / 10  Loss:  tensor(1.0487, grad_fn=<DivBackward0>)
Epoch:  219
Training  10 / 10  Loss:  tensor(1.0287, grad_fn=<DivBackward0>)
Epoch:  220
Training  10 / 10  Loss:  tensor(1.0624, grad_fn=<DivBackward0>)
Epoch:  221
Training  10 / 10  Loss:  tensor(1.0337, grad_fn=<DivBackward0>)
Epoch:  222
Training  10 / 10  Loss:  tensor(1.0376, grad_fn=<DivBackward0>)
Epoch:  223
Training  10 / 10  Loss:  tensor(1.0360, grad_fn=<DivBackward0>)
Epoch:  224
Training  10 / 10  Loss:  tensor(1.0167, grad_fn=<DivBackward0>)
Epoch:  225
Training  10 / 10  Loss:  tensor(1.0496, grad_fn=<DivBackward0>)
Epoch:  226
Training  10 / 10  Loss:  tensor(1.0477, grad_fn=<DivBackward0>)
Epoch:  227
Training  10 / 10  Loss:  tensor(1.0756, grad_fn=<DivBackward0>)
Epoch:  228

Training  10 / 10  Loss:  tensor(1.0123, grad_fn=<DivBackward0>)
Epoch:  323
Training  10 / 10  Loss:  tensor(1.0101, grad_fn=<DivBackward0>)
Epoch:  324
Training  10 / 10  Loss:  tensor(1.0334, grad_fn=<DivBackward0>)
Epoch:  325
Training  10 / 10  Loss:  tensor(1.0215, grad_fn=<DivBackward0>)
Epoch:  326
Training  10 / 10  Loss:  tensor(1.0559, grad_fn=<DivBackward0>)
Epoch:  327
Training  10 / 10  Loss:  tensor(1.0406, grad_fn=<DivBackward0>)
Epoch:  328
Training  10 / 10  Loss:  tensor(1.0136, grad_fn=<DivBackward0>)
Epoch:  329
Training  10 / 10  Loss:  tensor(1.0275, grad_fn=<DivBackward0>)
Epoch:  330
Training  10 / 10  Loss:  tensor(0.9907, grad_fn=<DivBackward0>)
Epoch:  331
Training  10 / 10  Loss:  tensor(0.9944, grad_fn=<DivBackward0>)
Epoch:  332
Training  10 / 10  Loss:  tensor(1.0239, grad_fn=<DivBackward0>)
Epoch:  333
Training  10 / 10  Loss:  tensor(1.0197, grad_fn=<DivBackward0>)
Epoch:  334
Training  10 / 10  Loss:  tensor(1.0410, grad_fn=<DivBackward0>)
Epoch:  335

Training  10 / 10  Loss:  tensor(0.9107, grad_fn=<DivBackward0>)
Epoch:  430
Training  10 / 10  Loss:  tensor(0.9132, grad_fn=<DivBackward0>)
Epoch:  431
Training  10 / 10  Loss:  tensor(0.9233, grad_fn=<DivBackward0>)
Epoch:  432
Training  10 / 10  Loss:  tensor(0.9109, grad_fn=<DivBackward0>)
Epoch:  433
Training  10 / 10  Loss:  tensor(0.9193, grad_fn=<DivBackward0>)
Epoch:  434
Training  10 / 10  Loss:  tensor(0.9136, grad_fn=<DivBackward0>)
Epoch:  435
Training  10 / 10  Loss:  tensor(0.9324, grad_fn=<DivBackward0>)
Epoch:  436
Training  10 / 10  Loss:  tensor(0.9243, grad_fn=<DivBackward0>)
Epoch:  437
Training  10 / 10  Loss:  tensor(0.9260, grad_fn=<DivBackward0>)
Epoch:  438
Training  10 / 10  Loss:  tensor(0.9136, grad_fn=<DivBackward0>)
Epoch:  439
Training  10 / 10  Loss:  tensor(0.9128, grad_fn=<DivBackward0>)
Epoch:  440
Training  10 / 10  Loss:  tensor(0.9136, grad_fn=<DivBackward0>)
Epoch:  441
Training  10 / 10  Loss:  tensor(0.9220, grad_fn=<DivBackward0>)
Epoch:  442

Training  10 / 10  Loss:  tensor(0.9043, grad_fn=<DivBackward0>)
Epoch:  537
Training  10 / 10  Loss:  tensor(0.9022, grad_fn=<DivBackward0>)
Epoch:  538
Training  10 / 10  Loss:  tensor(0.9043, grad_fn=<DivBackward0>)
Epoch:  539
Training  10 / 10  Loss:  tensor(0.9032, grad_fn=<DivBackward0>)
Epoch:  540
Training  10 / 10  Loss:  tensor(0.9051, grad_fn=<DivBackward0>)
Epoch:  541
Training  10 / 10  Loss:  tensor(0.8986, grad_fn=<DivBackward0>)
Epoch:  542
Training  10 / 10  Loss:  tensor(0.8980, grad_fn=<DivBackward0>)
Epoch:  543
Training  10 / 10  Loss:  tensor(0.8951, grad_fn=<DivBackward0>)
Epoch:  544
Training  10 / 10  Loss:  tensor(0.9007, grad_fn=<DivBackward0>)
Epoch:  545
Training  10 / 10  Loss:  tensor(0.8997, grad_fn=<DivBackward0>)
Epoch:  546
Training  10 / 10  Loss:  tensor(0.8955, grad_fn=<DivBackward0>)
Epoch:  547
Training  10 / 10  Loss:  tensor(0.8962, grad_fn=<DivBackward0>)
Epoch:  548
Training  10 / 10  Loss:  tensor(0.8983, grad_fn=<DivBackward0>)
Epoch:  549

Training  10 / 10  Loss:  tensor(0.8961, grad_fn=<DivBackward0>)
Epoch:  644
Training  10 / 10  Loss:  tensor(0.8930, grad_fn=<DivBackward0>)
Epoch:  645
Training  10 / 10  Loss:  tensor(0.8953, grad_fn=<DivBackward0>)
Epoch:  646
Training  10 / 10  Loss:  tensor(0.8928, grad_fn=<DivBackward0>)
Epoch:  647
Training  10 / 10  Loss:  tensor(0.8918, grad_fn=<DivBackward0>)
Epoch:  648
Training  10 / 10  Loss:  tensor(0.8926, grad_fn=<DivBackward0>)
Epoch:  649
Training  10 / 10  Loss:  tensor(0.8943, grad_fn=<DivBackward0>)
Epoch:  650
Training  10 / 10  Loss:  tensor(0.8945, grad_fn=<DivBackward0>)
Epoch:  651
Training  10 / 10  Loss:  tensor(0.8938, grad_fn=<DivBackward0>)
Epoch:  652
Training  10 / 10  Loss:  tensor(0.8941, grad_fn=<DivBackward0>)
Epoch:  653
Training  10 / 10  Loss:  tensor(0.8940, grad_fn=<DivBackward0>)
Epoch:  654
Training  10 / 10  Loss:  tensor(0.8951, grad_fn=<DivBackward0>)
Epoch:  655
Training  10 / 10  Loss:  tensor(0.8947, grad_fn=<DivBackward0>)
Epoch:  656

Training  10 / 10  Loss:  tensor(0.8902, grad_fn=<DivBackward0>)
Epoch:  751
Training  10 / 10  Loss:  tensor(0.8904, grad_fn=<DivBackward0>)
Epoch:  752
Training  10 / 10  Loss:  tensor(0.8897, grad_fn=<DivBackward0>)
Epoch:  753
Training  10 / 10  Loss:  tensor(0.8903, grad_fn=<DivBackward0>)
Epoch:  754
Training  10 / 10  Loss:  tensor(0.8921, grad_fn=<DivBackward0>)
Epoch:  755
Training  10 / 10  Loss:  tensor(0.8905, grad_fn=<DivBackward0>)
Epoch:  756
Training  10 / 10  Loss:  tensor(0.8903, grad_fn=<DivBackward0>)
Epoch:  757
Training  10 / 10  Loss:  tensor(0.8900, grad_fn=<DivBackward0>)
Epoch:  758
Training  10 / 10  Loss:  tensor(0.8909, grad_fn=<DivBackward0>)
Epoch:  759
Training  10 / 10  Loss:  tensor(0.8912, grad_fn=<DivBackward0>)
Epoch:  760
Training  10 / 10  Loss:  tensor(0.8909, grad_fn=<DivBackward0>)
Epoch:  761
Training  10 / 10  Loss:  tensor(0.8922, grad_fn=<DivBackward0>)
Epoch:  762
Training  10 / 10  Loss:  tensor(0.8922, grad_fn=<DivBackward0>)
Epoch:  763

Training  10 / 10  Loss:  tensor(0.8878, grad_fn=<DivBackward0>)
Epoch:  858
Training  10 / 10  Loss:  tensor(0.8878, grad_fn=<DivBackward0>)
Epoch:  859
Training  10 / 10  Loss:  tensor(0.8879, grad_fn=<DivBackward0>)
Epoch:  860
Training  10 / 10  Loss:  tensor(0.8877, grad_fn=<DivBackward0>)
Epoch:  861
Training  10 / 10  Loss:  tensor(0.8876, grad_fn=<DivBackward0>)
Epoch:  862
Training  10 / 10  Loss:  tensor(0.8876, grad_fn=<DivBackward0>)
Epoch:  863
Training  10 / 10  Loss:  tensor(0.8876, grad_fn=<DivBackward0>)
Epoch:  864
Training  10 / 10  Loss:  tensor(0.8875, grad_fn=<DivBackward0>)
Epoch:  865
Training  10 / 10  Loss:  tensor(0.8875, grad_fn=<DivBackward0>)
Epoch:  866
Training  10 / 10  Loss:  tensor(0.8875, grad_fn=<DivBackward0>)
Epoch:  867
Training  10 / 10  Loss:  tensor(0.8876, grad_fn=<DivBackward0>)
Epoch:  868
Training  10 / 10  Loss:  tensor(0.8876, grad_fn=<DivBackward0>)
Epoch:  869
Training  10 / 10  Loss:  tensor(0.8875, grad_fn=<DivBackward0>)
Epoch:  870

Training  10 / 10  Loss:  tensor(0.8873, grad_fn=<DivBackward0>)
Epoch:  965
Training  10 / 10  Loss:  tensor(0.8873, grad_fn=<DivBackward0>)
Epoch:  966
Training  10 / 10  Loss:  tensor(0.8873, grad_fn=<DivBackward0>)
Epoch:  967
Training  10 / 10  Loss:  tensor(0.8873, grad_fn=<DivBackward0>)
Epoch:  968
Training  10 / 10  Loss:  tensor(0.8873, grad_fn=<DivBackward0>)
Epoch:  969
Training  10 / 10  Loss:  tensor(0.8873, grad_fn=<DivBackward0>)
Epoch:  970
Training  10 / 10  Loss:  tensor(0.8873, grad_fn=<DivBackward0>)
Epoch:  971
Training  10 / 10  Loss:  tensor(0.8873, grad_fn=<DivBackward0>)
Epoch:  972
Training  10 / 10  Loss:  tensor(0.8873, grad_fn=<DivBackward0>)
Epoch:  973
Training  10 / 10  Loss:  tensor(0.8873, grad_fn=<DivBackward0>)
Epoch:  974
Training  10 / 10  Loss:  tensor(0.8873, grad_fn=<DivBackward0>)
Epoch:  975
Training  10 / 10  Loss:  tensor(0.8873, grad_fn=<DivBackward0>)
Epoch:  976
Training  10 / 10  Loss:  tensor(0.8873, grad_fn=<DivBackward0>)
Epoch:  977